In [54]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import Classification.config as cfg
import csv
import nltk
import random
import warnings
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')

In [2]:
data_set = []
for file in cfg.ALL_FILES:
    reader = csv.reader(open(file, 'r'), delimiter=';')
    for line in reader:
        data_set.append([line[0].lower(), line[1]])

words_bag = set(word for passage in data_set for word in nltk.tokenize.WordPunctTokenizer().tokenize(passage[0]))
#feature_sets = [({word: (word in nltk.tokenize.WordPunctTokenizer().tokenize(data[0])) for word in words_bag}, data[1]) for data in data_set]

In [3]:
feature_set_encoded= [([(word in nltk.tokenize.WordPunctTokenizer().tokenize(data[0])) for word in words_bag], data[1]) for data in data_set]


In [60]:
bag_frame_encoded = pd.DataFrame(feature_set_encoded)

x, y = bag_frame_encoded.iloc[:,:-1],bag_frame_encoded.iloc[:,-1]

x_frame = pd.DataFrame(x[0].tolist(), columns = words_bag)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_frame, y, test_size=0.2, random_state=3)

xg_class = xgb.XGBClassifier(max_depth=6, n_estimators=100, learning_rate=0.1, min_child_weight = 1, njobs=4)

In [61]:
xg_class.fit(x_train, y_train)
preds = xg_class.predict(x_test)
accuracy_score(preds, y_test)

0.930622009569378

In [51]:
from sklearn.grid_search import GridSearchCV
#Choose all predictors except target & IDcols
param_test1 =  {
 'learning_rate':[0.05, 0.1, 0.15],
 'n_estimators':[75, 100, 125],
 'max_depth': [6, 7],
 'min_child_weight': [1, 2]}
gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier(learning_rate = 0.1,
                                                      max_depth = 5,
                                                      random_state= 10,
                                                      min_child_weight = 1), 
param_grid = param_test1, n_jobs=4,iid=False, cv=5)
gsearch1.estimator.get_params()
gsearch1.fit(np.array(x_train), np.array(y_train))

gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.90538, std: 0.01378, params: {'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 75},
  mean: 0.91916, std: 0.01229, params: {'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 100},
  mean: 0.92754, std: 0.00719, params: {'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 125},
  mean: 0.89759, std: 0.01819, params: {'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 2, 'n_estimators': 75},
  mean: 0.89878, std: 0.01811, params: {'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 2, 'n_estimators': 100},
  mean: 0.89878, std: 0.01719, params: {'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 2, 'n_estimators': 125},
  mean: 0.91436, std: 0.01087, params: {'learning_rate': 0.05, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 75},
  mean: 0.92694, std: 0.00902, params: {'learning_rate': 0.05, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 100},
  mean: 0.9

In [23]:
estimator = xgb.XGBClassifier()
estimator.get_params().keys()

dict_keys(['base_score', 'booster', 'colsample_bylevel', 'colsample_bytree', 'gamma', 'learning_rate', 'max_delta_step', 'max_depth', 'min_child_weight', 'missing', 'n_estimators', 'n_jobs', 'nthread', 'objective', 'random_state', 'reg_alpha', 'reg_lambda', 'scale_pos_weight', 'seed', 'silent', 'subsample'])

In [17]:
test_frame = pd.DataFrame(gsearch1.grid_scores_)

In [18]:
test_frame

,parameters,mean_validation_score,cv_validation_scores
0,"{'learning_rate': 0.05, 'max_depth': 3, 'n_est...",0.917358,"[0.9283582089552239, 0.9131736526946108, 0.925..."
1,"{'learning_rate': 0.05, 'max_depth': 3, 'n_est...",0.926340,"[0.9343283582089552, 0.9191616766467066, 0.940..."
2,"{'learning_rate': 0.05, 'max_depth': 3, 'n_est...",0.929934,"[0.9343283582089552, 0.9221556886227545, 0.946..."
3,"{'learning_rate': 0.05, 'max_depth': 5, 'n_est...",0.930533,"[0.9343283582089552, 0.9311377245508982, 0.946..."
4,"{'learning_rate': 0.05, 'max_depth': 5, 'n_est...",0.929933,"[0.9343283582089552, 0.9341317365269461, 0.946..."
5,"{'learning_rate': 0.05, 'max_depth': 5, 'n_est...",0.931132,"[0.9343283582089552, 0.937125748502994, 0.9461..."
6,"{'learning_rate': 0.05, 'max_depth': 8, 'n_est...",0.929933,"[0.9343283582089552, 0.9341317365269461, 0.946..."
7,"{'learning_rate': 0.05, 'max_depth': 8, 'n_est...",0.930533,"[0.9313432835820895, 0.937125748502994, 0.9461..."
8,"{'learning_rate': 0.05, 'max_depth': 8, 'n_est...",0.928731,"[0.9343283582089552, 0.9311377245508982, 0.946..."
9,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",0.931733,"[0.9343283582089552, 0.9311377245508982, 0.949..."


In [134]:
test['Pred']=preds

In [108]:
depth = [3,5, 10]
esti = [50, 100, 300, 1000]
learning_rate = [0.05, 0.2, 0.4]

for d in depth:
    for e in esti:
        for l in learning_rate:
            xg_class = xgb.XGBClassifier(max_depth=d, n_estimators=e, learning_rate=l)
            xg_class.fit(x_train, y_train)
            preds = xg_class.predict(x_test)
            print('Depth:', d, 'Estimators:', e, 'Learning Rate:', l, 'Accuracy', accuracy_score(preds, y_test))

In [145]:
len(preds)*937799043062201

392000000000000018

In [150]:
wrong = [(index, test['Pred'][index])  for index in test.index if test[1][index] != test['Pred'][index]]

In [157]:
wrong_2 = [(index, test['Pred'][index])  for index in test.index if test[1][index] != test['Pred'][index]]

In [161]:
len(wrong_2)

34

In [153]:
for i, pred in wrong:
    print(i, data_set[i], 'pred:', pred)

678 ['hey crew, was empfehlt ihr in sachen kamera- und videoschnitt-apps auf android?', '0'] pred: 1
1663 ['ist das so einer, wo man nur am griff drehen muss?', '1'] pred: 0
209 ['was müsst ihr machen?', '0'] pred: 1
327 ['help, worum gehts hier eigentlich?', '0'] pred: 1
1888 ['um wieviele packungen handelt es sich bei diesem angebot?', '0'] pred: 1
1188 ['wisst ihr warum deutschland keine bananenrepublik ist?', '1'] pred: 0
108 ['war er jetzt da oder kam er zu spät?', '0'] pred: 1
1241 ['vollkornbrot, mehrkornbrot, sonnenblumenkernbrot, mohnbrot oder normales brot?', '0'] pred: 1
235 ['woran liegt das?', '0'] pred: 1
427 ['weißt du aber, was der gemacht hat?', '1'] pred: 0
1556 ['hat jemand welche?', '1'] pred: 0
682 ['seh ich aus wie ein auto?', '1'] pred: 0
1164 ['was wünscht ihr euch vor 20.00uhr?', '0'] pred: 1
1586 ['wieviele messungen kann dieses gerät speichern?', '0'] pred: 1
911 ['was geschieht bei euch in der region, wovon wir nichts mitbekommen?', '0'] pred: 1
856 ['weiß j

In [71]:
from sklearn.metrics import accuracy_score

In [99]:
accuracy = accuracy_score(preds, y_test)

In [100]:
accuracy

0.9258373205741627

In [101]:
data_dmatrix = xgb.DMatrix(data=x_frame,label=y)

TypeError: must be real number, not str

In [75]:
params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

[15:39:17] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[15:39:17] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[15:39:17] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[15:39:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[15:39:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[15:39:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[15:39:18] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 

[15:39:19] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[15:39:19] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[15:39:19] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[15:39:19] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[15:39:19] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[15:39:19] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[15:39:19] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 

[15:39:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[15:39:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[15:39:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[15:39:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[15:39:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[15:39:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[15:39:21] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 

In [76]:
cv_results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,0.450754,5.551115e-17,0.450754,5.551115e-17
1,0.406429,0.000000e+00,0.406429,0.000000e+00
2,0.366533,0.000000e+00,0.366533,0.000000e+00
3,0.330624,0.000000e+00,0.330624,0.000000e+00
4,0.298303,0.000000e+00,0.298303,0.000000e+00
